In [ ]:
import zipfile
import pandas as pd
import polars as pl
import geopandas as gpd
import requests
import os
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import re
import pycountry
import geodata
from tqdm import tqdm
from zipfile import ZipFile
from geopy.geocoders import Nominatim
import time
import urllib.request
import pycountry
from google.colab import files
import matplotlib.pyplot as plt
import contextily as ctx
import fiona
import tools
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import unicodedata
import pickle
import osmnx as ox
from concurrent.futures import ThreadPoolExecutor, as_completed
from shapely.geometry import Point
from matplotlib.ticker import MaxNLocator
import pandas as pd
import numpy as np
from itertools import product

In [ ]:
FONTSIZE = 18
params = {
         'axes.labelsize': FONTSIZE,
         'axes.titlesize': FONTSIZE,
         'xtick.labelsize':FONTSIZE,
         'legend.fontsize':FONTSIZE,
         'ytick.labelsize':FONTSIZE}

plt.rcParams.update(params)

image_folder = '/content/Linking-Plastic-to-Dengue-Incidence/plots'
data_folder = '/content/Linking-Plastic-to-Dengue-Incidence/data'
os.makedirs(image_folder, exist_ok=True)
os.makedirs(data_folder, exist_ok=True)

# Load GADM dataset

In [ ]:
# Load the file
gadm_subset = gpd.read_file(f"{data_folder}/gadm_subset_gdf.gpkg")

# Reproject to World Mollweide (suitable for global area-preserving calculations)
gdf_projected = gadm_subset.to_crs(epsg=3857)

# Compute centroids in projected space
gdf_projected["centroid"] = gdf_projected.geometry.centroid

# Create GeoDataFrame from centroids and reproject back to EPSG:4326
gdf_projected = gdf_projected.set_geometry("centroid")
gdf_centroids = gdf_projected.to_crs(epsg=4326)

# Extract Point geometry back into original GeoDataFrame
gadm_subset["centroid"] = gdf_centroids.geometry

gadm_subset.head()

,adm_0_name,adm_1_name,adm_2_name,GID_2,geometry,centroid
0,afghanistan,badakhshan,baharak,AFG.1.1_1,"MULTIPOLYGON (((71.41149 36.55717, 71.40954 36...",POINT (71.10439 37.0221)
1,afghanistan,badakhshan,darwaz,AFG.1.2_1,"MULTIPOLYGON (((71.2762 38.00465, 71.27578 38....",POINT (70.93919 38.21125)
2,afghanistan,badakhshan,fayzabad,AFG.1.3_1,"MULTIPOLYGON (((70.78272 37.27678, 70.78635 37...",POINT (70.47054 37.11547)
3,afghanistan,badakhshan,ishkashim,AFG.1.4_1,"MULTIPOLYGON (((71.41149 36.55717, 71.40091 36...",POINT (71.42752 36.80869)
4,afghanistan,badakhshan,jurm,AFG.1.5_1,"MULTIPOLYGON (((70.71236 37.07621, 70.73582 37...",POINT (70.82746 36.58068)


# Load Atmosphere Data

In [ ]:
atmosphere_data = pd.read_excel("data_plastic_dengue_original.xls", header=1)
atmosphere_data.tail()

,LONGITUDE,LATITUDE,t2m_bilinear,t2m_bilinear.1,tp_bilinear,tp_bilinear.1,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
47981,33.738098,35.200216,21.530004,21.121913,88.100108,82.126120,NaN,NaN,NaN,NaN
47982,33.237251,35.304342,20.736938,19.617941,130.313644,120.300582,NaN,NaN,NaN,NaN
47983,32.951355,35.176131,19.635524,20.513656,157.895394,147.605507,NaN,NaN,NaN,NaN
47984,34.103874,35.446732,21.091701,NaN,124.328840,NaN,NaN,NaN,NaN,NaN
47985,33.429809,35.173368,21.002876,20.416136,107.899790,102.116520,NaN,NaN,NaN,NaN


In [ ]:
# Load the Excel file
atmosphere_data = pd.read_excel("data_plastic_dengue_original.xls", header=1)

# Drop completely empty or unused columns
atmosphere_data = atmosphere_data.dropna(axis=1, how='all')
atmosphere_data = atmosphere_data.drop(columns=["Unnamed: 8", "Unnamed: 9"])

# Create Point geometries
centroid = [Point(xy) for xy in zip(atmosphere_data["LONGITUDE"], atmosphere_data["LATITUDE"])]

# Convert to GeoDataFrame with CRS set
atmosphere_data = gpd.GeoDataFrame(atmosphere_data, geometry=centroid, crs="EPSG:4326")

# Project to EPSG:3857
atmosphere_data_projected = atmosphere_data.to_crs(epsg=3857)

# Compute centroids (in projected space)
atmosphere_data_projected["centroid"] = atmosphere_data_projected.geometry.centroid

# Set geometry to centroid and reproject back to EPSG:4326
atmosphere_data_centroids = atmosphere_data_projected.set_geometry("centroid").to_crs(epsg=4326)

# Clean up and rename columns
atmosphere_data = atmosphere_data_centroids.drop(columns=["LONGITUDE", "LATITUDE", "geometry"])
atmosphere_data.columns = [
    "ERA5_t2m", "ERA5land_t2m", "ERA5_tp", "ERA5land_tp", "centroid"
]

# Final shape and preview
print(atmosphere_data.shape)
atmosphere_data.head()

(47986, 5)


,ERA5_t2m,ERA5land_t2m,ERA5_tp,ERA5land_tp,centroid
0,1.091406,0.557270,188.098434,179.431148,POINT (71.10453 37.02116)
1,-1.447696,-5.208449,412.987901,397.445158,POINT (70.9392 38.21097)
2,8.561153,9.135770,197.963614,184.069205,POINT (70.47056 37.11494)
3,-3.465137,-7.374354,186.709560,179.852878,POINT (71.42761 36.80787)
4,-1.253492,2.983343,217.539726,219.460520,POINT (70.8274 36.5798)


In [29]:
# If not already in memory:
# atmosphere_data = pd.read_csv("your_file.csv")  # or however you created it

# Ensure the columns are numeric (coerce errors just in case)
atmosphere_data["ERA5_t2m"] = pd.to_numeric(atmosphere_data["ERA5_t2m"], errors="coerce")
atmosphere_data["ERA5_tp"]  = pd.to_numeric(atmosphere_data["ERA5_tp"],  errors="coerce")

# Drop rows where either variable is missing (optional but typical)
subset = atmosphere_data[["ERA5_t2m", "ERA5_tp"]].dropna()

# Calculate SDs (sample SD by default; add ddof=0 for population SD)
sd_t2m = subset["ERA5_t2m"].std()      # or .std(ddof=1)
sd_tp  = subset["ERA5_tp"].std()

print(f"SD of ERA5_t2m: {sd_t2m:.3f}")
print(f"SD of ERA5_tp:  {sd_tp:.3f}")


SD of ERA5_t2m: 7.699
SD of ERA5_tp:  314.988


In [ ]:
# Reproject both to EPSG:3857 (meters)
gadm_subset_proj = gadm_subset.to_crs(epsg=3857)
atmosphere_data_proj = atmosphere_data.to_crs(epsg=3857)

# Join with nearest
atmosphere_with_admin = gpd.sjoin_nearest(
    gadm_subset_proj,
    atmosphere_data_proj,
    how="left",
    distance_col="distance_to_weather_point"
)

# Filter to only accept matches within 50 meters
atmosphere_with_admin = atmosphere_with_admin[atmosphere_with_admin["distance_to_weather_point"] <= 50]

# Convert back to EPSG:4326 if needed
atmosphere_with_admin = atmosphere_with_admin.to_crs(epsg=4326)
atmosphere_with_admin = atmosphere_with_admin.drop(columns=[
    "index_right", "distance_to_weather_point"
])
atmosphere_with_admin.head()

,adm_0_name,adm_1_name,adm_2_name,GID_2,geometry,centroid,ERA5_t2m,ERA5land_t2m,ERA5_tp,ERA5land_tp
0,afghanistan,badakhshan,baharak,AFG.1.1_1,"MULTIPOLYGON (((71.41149 36.55717, 71.40954 36...",POINT (71.10439 37.0221),1.091406,0.557270,188.098434,179.431148
1,afghanistan,badakhshan,darwaz,AFG.1.2_1,"MULTIPOLYGON (((71.2762 38.00465, 71.27578 38....",POINT (70.93919 38.21125),-1.447696,-5.208449,412.987901,397.445158
2,afghanistan,badakhshan,fayzabad,AFG.1.3_1,"MULTIPOLYGON (((70.78272 37.27678, 70.78635 37...",POINT (70.47054 37.11547),8.561153,9.135770,197.963614,184.069205
3,afghanistan,badakhshan,ishkashim,AFG.1.4_1,"MULTIPOLYGON (((71.41149 36.55717, 71.40091 36...",POINT (71.42752 36.80869),-3.465137,-7.374354,186.709560,179.852878
4,afghanistan,badakhshan,jurm,AFG.1.5_1,"MULTIPOLYGON (((70.71236 37.07621, 70.73582 37...",POINT (70.82746 36.58068),-1.253492,2.983343,217.539726,219.460520


# Load merged plastic dengue data full with NaNs


In [ ]:
# # Load the file
# plastic_dengue_merged_full = gpd.read_file("/content/plastic_dengue_merged_full_gdf.gpkg")

# # Reproject to World Mollweide (suitable for global area-preserving calculations)
# gdf_projected = plastic_dengue_merged_full.to_crs(epsg=3857)

# # Compute centroids in projected space
# gdf_projected["centroid"] = gdf_projected.geometry.centroid

# # Create GeoDataFrame from centroids and reproject back to EPSG:4326
# gdf_projected = gdf_projected.set_geometry("centroid")
# gdf_centroids = gdf_projected.to_crs(epsg=4326)

# # Extract Point geometry back into original GeoDataFrame
# plastic_dengue_merged_full["centroid"] = gdf_centroids.geometry

# plastic_dengue_merged_full.head()

In [ ]:
# plastic_dengue_merged_full.shape

In [ ]:
# plastic_dengue_atmosphere_full = plastic_dengue_merged_full.merge(
#     atmosphere_with_admin[["centroid",	"ERA5_t2m",	"ERA5land_t2m",	"ERA5_tp", "ERA5land_tp"]],
#     on="centroid",
#     how='inner'
# )
# plastic_dengue_atmosphere_full.head()

In [ ]:
# plastic_dengue_atmosphere_full.shape

In [ ]:
# plastic_dengue_atmosphere_full.isna().sum()[plastic_dengue_atmosphere_full.isna().sum()>0]

# Load merged Dengue Plastic data with no NaNs

In [ ]:
# Load the file
plastic_dengue_merged = gpd.read_file(f"{data_folder}/plastic_dengue_merged_gdf.gpkg")

# Reproject to World Mollweide (suitable for global area-preserving calculations)
gdf_projected = plastic_dengue_merged.to_crs(epsg=3857)

# Compute centroids in projected space
gdf_projected["centroid"] = gdf_projected.geometry.centroid

# Create GeoDataFrame from centroids and reproject back to EPSG:4326
gdf_projected = gdf_projected.set_geometry("centroid")
gdf_centroids = gdf_projected.to_crs(epsg=4326)

# Extract Point geometry back into original GeoDataFrame
plastic_dengue_merged["centroid"] = gdf_centroids.geometry

plastic_dengue_merged.head()

,adm_0_name,ISO3,Income_cat,adm_1_name,adm_2_name,Unique_ID,GID_2,Population_2020,Rural_share,Deg_Urban_Lv1,...,plas_disp_ob_em_pct_em,plas_inf_deb_em_pct_em,plas_inf_ob_em_pct_em,plas_form_deb_em_pct_em,plas_form_ob_em_pct_em,people_no_col,people_no_col_pct_pop,mean_dengue_2014_2023,geometry,centroid
0,argentina,ARG,UMC,buenos aires,almirante brown,ARG.1.4_1,ARG.1.4_1,560097.564993,0.003252,3,...,0.021239,0.058559,0.090610,0.0,0.0,34990.319466,6.247183,61.5,"MULTIPOLYGON (((-58.39741 -34.91209, -58.40034...",POINT (-58.36831 -34.83864)
1,argentina,ARG,UMC,chaco,almirante brown,ARG.1.4_1,ARG.3.1_1,560097.564993,0.003252,3,...,0.021239,0.058559,0.090610,0.0,0.0,34990.319466,6.247183,657.0,"MULTIPOLYGON (((-60.79854 -26.42871, -60.91188...",POINT (-61.9843 -25.74921)
2,argentina,ARG,UMC,buenos aires,avellaneda,ARG.1.7_1,ARG.1.7_1,399825.426948,0.001051,3,...,0.000000,0.046689,0.073099,0.0,0.0,9760.323288,2.441146,360.0,"MULTIPOLYGON (((-58.33411 -34.73055, -58.33783...",POINT (-58.34009 -34.68142)
3,argentina,ARG,UMC,rio negro,avellaneda,ARG.1.7_1,ARG.16.2_1,399825.426948,0.001051,3,...,0.000000,0.046689,0.073099,0.0,0.0,9760.323288,2.441146,2.0,"MULTIPOLYGON (((-66.64465 -38.53105, -66.64307...",POINT (-66.22107 -39.41774)
4,argentina,ARG,UMC,buenos aires,azul,ARG.1.9_1,ARG.1.9_1,74203.968586,0.213579,3,...,0.316862,0.215094,0.412950,0.0,0.0,12513.923624,16.864224,2.0,"MULTIPOLYGON (((-59.49092 -36.23094, -59.2936 ...",POINT (-59.69994 -36.79136)


In [ ]:
plastic_dengue_merged.shape

(6480, 67)

In [ ]:
plastic_dengue_atmosphere = plastic_dengue_merged.merge(
    atmosphere_with_admin[["centroid",	"ERA5_t2m",	"ERA5land_t2m",	"ERA5_tp", "ERA5land_tp"]],
    on='centroid',
    how='inner'
)
plastic_dengue_atmosphere.head()

,adm_0_name,ISO3,Income_cat,adm_1_name,adm_2_name,Unique_ID,GID_2,Population_2020,Rural_share,Deg_Urban_Lv1,...,plas_form_ob_em_pct_em,people_no_col,people_no_col_pct_pop,mean_dengue_2014_2023,geometry,centroid,ERA5_t2m,ERA5land_t2m,ERA5_tp,ERA5land_tp
0,argentina,ARG,UMC,buenos aires,almirante brown,ARG.1.4_1,ARG.1.4_1,560097.564993,0.003252,3,...,0.0,34990.319466,6.247183,61.5,"MULTIPOLYGON (((-58.39741 -34.91209, -58.40034...",POINT (-58.36831 -34.83864),17.314585,17.274513,279.710384,265.970576
1,argentina,ARG,UMC,buenos aires,avellaneda,ARG.1.7_1,ARG.1.7_1,399825.426948,0.001051,3,...,0.0,9760.323288,2.441146,360.0,"MULTIPOLYGON (((-58.33411 -34.73055, -58.33783...",POINT (-58.34009 -34.68142),17.630055,17.567303,293.873937,276.584939
2,argentina,ARG,UMC,rio negro,avellaneda,ARG.1.7_1,ARG.16.2_1,399825.426948,0.001051,3,...,0.0,9760.323288,2.441146,2.0,"MULTIPOLYGON (((-66.64465 -38.53105, -66.64307...",POINT (-66.22107 -39.41774),15.835646,16.029452,114.102148,109.387430
3,argentina,ARG,UMC,buenos aires,azul,ARG.1.9_1,ARG.1.9_1,74203.968586,0.213579,3,...,0.0,12513.923624,16.864224,2.0,"MULTIPOLYGON (((-59.49092 -36.23094, -59.2936 ...",POINT (-59.69994 -36.79136),14.879260,15.174602,272.429956,257.711472
4,argentina,ARG,UMC,buenos aires,bahia blanca,ARG.1.10_1,ARG.1.10_1,322362.958835,0.034462,3,...,0.0,26344.491008,8.172307,3.0,"MULTIPOLYGON (((-62.21736 -38.83569, -62.21625...",POINT (-62.16959 -38.58023),15.509579,15.954112,200.357109,186.168997


In [ ]:
plastic_dengue_atmosphere.shape

(6473, 71)

In [ ]:
plastic_dengue_atmosphere.isna().sum()[plastic_dengue_atmosphere.isna().sum()>0]

,0
ERA5land_t2m,10
ERA5land_tp,10


In [ ]:
#  Convert to GeoDataFrame (specify the geometry column)
plastic_dengue_atmosphere_gdf = plastic_dengue_atmosphere.drop(columns=["centroid"])
plastic_dengue_atmosphere_gdf = gpd.GeoDataFrame(plastic_dengue_atmosphere_gdf, geometry="geometry")

# Save as GeoPackage
plastic_dengue_atmosphere_gdf.to_file(f"{data_folder}/plastic_dengue_atmosphere_gdf.gpkg", driver="GPKG")

print(" Saved plastic_dengue_atmosphere_gdf as 'plastic_dengue_atmosphere_gdf.gpkg'")

 Saved plastic_dengue_atmosphere_gdf as 'plastic_dengue_atmosphere_gdf.gpkg'


In [ ]:
plastic_dengue_atmosphere_gdf[['adm_0_name', 'adm_1_name', 'adm_2_name', 'plas_litter_em', 'ERA5_t2m', 'mean_dengue_2014_2023']].head()

,adm_0_name,adm_1_name,adm_2_name,plas_litter_em,ERA5_t2m,mean_dengue_2014_2023
0,argentina,buenos aires,almirante brown,25.713250,17.314585,61.5
1,argentina,buenos aires,avellaneda,17.759832,17.630055,360.0
2,argentina,rio negro,avellaneda,17.759832,15.835646,2.0
3,argentina,buenos aires,azul,16.471572,14.879260,2.0
4,argentina,buenos aires,bahia blanca,35.394667,15.509579,3.0


In [ ]:
plastic_dengue_atmosphere_gdf[plastic_dengue_atmosphere_gdf.adm_0_name=="united states of america"]

,adm_0_name,ISO3,Income_cat,adm_1_name,adm_2_name,Unique_ID,GID_2,Population_2020,Rural_share,Deg_Urban_Lv1,...,plas_form_deb_em_pct_em,plas_form_ob_em_pct_em,people_no_col,people_no_col_pct_pop,mean_dengue_2014_2023,geometry,ERA5_t2m,ERA5land_t2m,ERA5_tp,ERA5land_tp
6441,united states of america,USA,HIC,texas,dallas,USA.1.24_1,USA.44.57_1,4.110051e+04,0.601441,1,...,0.0,0.0,0.0,0.0,2.0,"MULTIPOLYGON (((-96.8462 32.98834, -96.51854 3...",19.598701,19.543202,348.270731,328.809008
6442,united states of america,USA,HIC,florida,monroe,USA.1.50_1,USA.10.44_1,2.150496e+04,1.000000,1,...,0.0,0.0,0.0,0.0,124.0,"MULTIPOLYGON (((-81.96267 24.52263, -81.96264 ...",25.206234,24.874536,335.689249,302.276277
6443,united states of america,USA,HIC,arizona,maricopa,USA.3.8_1,USA.3.8_1,4.724025e+06,0.040602,3,...,0.0,0.0,0.0,0.0,2.0,"MULTIPOLYGON (((-111.49599 34.00005, -111.4898...",23.911275,23.984137,67.011952,63.041795
6444,united states of america,USA,HIC,florida,polk,USA.4.57_1,USA.10.53_1,2.077056e+04,1.000000,1,...,0.0,0.0,0.0,0.0,2.0,"MULTIPOLYGON (((-81.56371 27.64642, -81.61702 ...",23.304204,22.890821,422.870340,388.665996
6445,united states of america,USA,HIC,california,los angeles,USA.5.19_1,USA.5.19_1,1.011242e+07,0.011412,3,...,0.0,0.0,0.0,0.0,16.0,"MULTIPOLYGON (((-118.43278 32.8025, -118.43273...",17.380851,15.972624,123.702729,111.877398
6446,united states of america,USA,HIC,florida,orange,USA.5.30_1,USA.10.48_1,3.153842e+06,0.004119,3,...,0.0,0.0,0.0,0.0,2.0,"MULTIPOLYGON (((-80.96403 28.60547, -80.95225 ...",22.938966,22.602204,422.676951,389.201178
6447,united states of america,USA,HIC,north carolina,orange,USA.5.30_1,USA.34.68_1,3.153842e+06,0.004119,3,...,0.0,0.0,0.0,0.0,2.0,"MULTIPOLYGON (((-79.2521 35.8768, -79.25882 35...",16.259243,15.912982,399.180642,376.980036
6448,united states of america,USA,HIC,california,san bernardino,USA.5.36_1,USA.5.36_1,2.339231e+06,0.081576,3,...,0.0,0.0,0.0,0.0,2.0,"MULTIPOLYGON (((-114.13727 34.30508, -114.1366...",21.595526,21.168162,32.009068,29.620892
6449,united states of america,USA,HIC,california,san diego,USA.5.37_1,USA.5.37_1,3.471947e+06,0.058134,3,...,0.0,0.0,0.0,0.0,2.0,"MULTIPOLYGON (((-117.23302 32.77584, -117.2329...",16.977832,16.703803,134.842658,120.775316
6450,united states of america,USA,HIC,florida,broward,USA.10.6_1,USA.10.6_1,1.942532e+06,0.003116,3,...,0.0,0.0,0.0,0.0,10.0,"MULTIPOLYGON (((-80.00014 26.00014, -79.99986 ...",24.897890,24.282214,515.357113,463.710040


In [ ]:
plastic_dengue_merged[plastic_dengue_merged.adm_0_name=="united states of america"]

,adm_0_name,ISO3,Income_cat,adm_1_name,adm_2_name,Unique_ID,GID_2,Population_2020,Rural_share,Deg_Urban_Lv1,...,plas_disp_ob_em_pct_em,plas_inf_deb_em_pct_em,plas_inf_ob_em_pct_em,plas_form_deb_em_pct_em,plas_form_ob_em_pct_em,people_no_col,people_no_col_pct_pop,mean_dengue_2014_2023,geometry,centroid
6448,united states of america,USA,HIC,texas,dallas,USA.1.24_1,USA.44.57_1,4.110051e+04,0.601441,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,"MULTIPOLYGON (((-96.8462 32.98834, -96.51854 3...",POINT (-96.77933 32.76827)
6449,united states of america,USA,HIC,florida,monroe,USA.1.50_1,USA.10.44_1,2.150496e+04,1.000000,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,124.0,"MULTIPOLYGON (((-81.96267 24.52263, -81.96264 ...",POINT (-81.06646 25.40449)
6450,united states of america,USA,HIC,arizona,maricopa,USA.3.8_1,USA.3.8_1,4.724025e+06,0.040602,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,"MULTIPOLYGON (((-111.49599 34.00005, -111.4898...",POINT (-112.48924 33.35202)
6451,united states of america,USA,HIC,florida,polk,USA.4.57_1,USA.10.53_1,2.077056e+04,1.000000,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,"MULTIPOLYGON (((-81.56371 27.64642, -81.61702 ...",POINT (-81.69781 27.94929)
6452,united states of america,USA,HIC,california,los angeles,USA.5.19_1,USA.5.19_1,1.011242e+07,0.011412,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,"MULTIPOLYGON (((-118.43278 32.8025, -118.43273...",POINT (-118.2266 34.324)
6453,united states of america,USA,HIC,florida,orange,USA.5.30_1,USA.10.48_1,3.153842e+06,0.004119,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,"MULTIPOLYGON (((-80.96403 28.60547, -80.95225 ...",POINT (-81.32372 28.5145)
6454,united states of america,USA,HIC,north carolina,orange,USA.5.30_1,USA.34.68_1,3.153842e+06,0.004119,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,"MULTIPOLYGON (((-79.2521 35.8768, -79.25882 35...",POINT (-79.12244 36.06097)
6455,united states of america,USA,HIC,california,san bernardino,USA.5.36_1,USA.5.36_1,2.339231e+06,0.081576,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,"MULTIPOLYGON (((-114.13727 34.30508, -114.1366...",POINT (-116.18056 34.84721)
6456,united states of america,USA,HIC,california,san diego,USA.5.37_1,USA.5.37_1,3.471947e+06,0.058134,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,"MULTIPOLYGON (((-117.23302 32.77584, -117.2329...",POINT (-116.73255 33.03694)
6457,united states of america,USA,HIC,florida,broward,USA.10.6_1,USA.10.6_1,1.942532e+06,0.003116,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,"MULTIPOLYGON (((-80.00014 26.00014, -79.99986 ...",POINT (-80.48771 26.15219)


In [ ]:
plastic_dengue_atmosphere_gdf.shape

(6473, 70)

In [ ]:
plastic_dengue_atmosphere_gdf.adm_0_name.unique()

array(['argentina', 'brazil', 'china', 'colombia', 'costa rica',
       'pakistan', 'peru', 'united states of america'], dtype=object)